In [3]:
import os
import pandas as pd
import cv2
import shutil
import base64
import re
global df
df= pd.DataFrame()


In [59]:
import os
import pandas as pd
import cv2
import shutil
import base64
import re
global df
df= pd.DataFrame()



# function to convert image to byte string
def image_to_byte_string(image):
    _, buffer = cv2.imencode('.jpg', image)
    byte_string = base64.b64encode(buffer).decode()
    return byte_string

# function to convert video to image
def vid2frames(video_chunk_num):
    # path to input video
    part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
    part_frame_folder_path = re.search(r"(p\d+)", video_chunk_num).group()
    part_frame_path = os.path.join(part_fpath,part_frame_folder_path)
    main_video_path =f'{os.path.join(part_fpath, video_chunk_num)}'
    
    print(part_frame_path)
    os.makedirs(part_frame_path, exist_ok = True)
   


    video_path = main_video_path
    print(video_path)


    # capture video
    cap = cv2.VideoCapture(video_path)
    print(cap)
    # loop through frames and save as images
    frame_count = 1
    while cap.isOpened():
        ret, frame = cap.read()
        print("h1")
        if ret:
            
            # save frame as image
            cv2.imwrite(f'{part_frame_path}/{video_chunk_num}-frame{frame_count}.jpg', frame)
            print(f'{video_chunk_num}/frame{frame_count}.jpg')
            frame_count += 1
        else:
            break

    # release video capture and destroy window
    cap.release()
    cv2.destroyAllWindows()

# function to split video into 10-second chunks
def split_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    chunk_size = int(fps * 10)  # 10-second chunks
    chunks = [(i, min(i+chunk_size, total_frames)) for i in range(0, total_frames, chunk_size)]
    frame_indices = [int(i*fps)+1 for i in range(10)]  # indices of frames to keep in each chunk
    return chunks, frame_indices

# function to create dataframe with video data
def create_dataframe(video_urls):
    data = {'video_id': [], 'video_url': [], 'video_parts': [], 'frame_rate': []}
    for video_url in video_urls:
        video_id = video_url.split('=')[-1]
        video_path = f'{video_id}.mp4'
        #os.system(f'youtube-dl {video_url} -o "{video_path}"')  # download video
        chunks, frame_indices = split_video(video_path)
        for i, chunk in enumerate(chunks):
            start, end = chunk
            part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
            os.makedirs(part_fpath, exist_ok = True)
            part_path = f'{os.path.join(part_fpath, f"{video_id}-p{i+1}.mp4")}'
            #print(f"DJ={part_path}")
            #print(f"DJ2={part_fpath}")
            video_chunk= f'{video_id}-p{i+1}.mp4'
            cap = cv2.VideoCapture(video_path)
            out = cv2.VideoWriter(part_path, cv2.VideoWriter_fourcc(*'mp4v'), 2, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
            
            for j in range(start, end):
                ret, frame = cap.read()
                if not ret:
                    break
                if j in frame_indices:
                    out.write(frame)
            out.release()
            
            data['video_id'].append(video_id)
            data['video_url'].append(video_url)
            data['video_parts'].append(video_chunk)
            data['frame_rate'].append(len(frame_indices))  # 2 frames per second
    df = pd.DataFrame(data)
    return df










# create dataframe
video_urls = ["https://www.youtube.com/watch?v=JDwduqfuHjc"]
df = create_dataframe(video_urls)
#df = vid2string(df)

save_path = r"C:\Users\SkyLake\Documents\Python 2.0\Proj02b\HEXA"

# save video chunks in target directory
#for part_path in df['video_parts']:
    #shutil.copy2(part_path, save_path)
    #shutil.move(part_path, save_path)
    #print(f'moved {part_path} to {save_path}')

# print dataframe
print(df)


      video_id                                    video_url  \
0  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
1  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
2  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
3  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
4  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
5  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
6  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   

          video_parts  frame_rate  
0  JDwduqfuHjc-p1.mp4          10  
1  JDwduqfuHjc-p2.mp4          10  
2  JDwduqfuHjc-p3.mp4          10  
3  JDwduqfuHjc-p4.mp4          10  
4  JDwduqfuHjc-p5.mp4          10  
5  JDwduqfuHjc-p6.mp4          10  
6  JDwduqfuHjc-p7.mp4          10  


In [35]:
df

,video_id,video_url,video_parts,frame_rate,frame_1,frame_2,frame_3,frame_4,frame_5,frame_6,frame_7,frame_8,frame_9,frame_10
0,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p1.mp4,10,,,,,,,,,,
1,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p2.mp4,10,,,,,,,,,,
2,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p3.mp4,10,,,,,,,,,,
3,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p4.mp4,10,,,,,,,,,,
4,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p5.mp4,10,,,,,,,,,,
5,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p6.mp4,10,,,,,,,,,,
6,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p7.mp4,10,,,,,,,,,,


In [ ]:
import base64

# function to convert image to byte string
def image_to_byte_string(image):
    _, buffer = cv2.imencode('.jpg', image)
    byte_string = base64.b64encode(buffer).decode()
    return byte_string




In [66]:
vid2frames("JDwduqfuHjc-p2.mp4")

c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p2
c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\JDwduqfuHjc-p2.mp4
< cv2.VideoCapture 000001E234E31BD0>
h1
JDwduqfuHjc-p2.mp4/frame1.jpg
h1
JDwduqfuHjc-p2.mp4/frame2.jpg
h1


In [24]:
import os
import pandas as pd
import cv2

# function to split video into 10-second chunks
def split_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    chunk_size = int(fps * 10)  # 10-second chunks
    chunks = [(i, min(i+chunk_size, total_frames)) for i in range(0, total_frames, chunk_size)]
    frame_indices = [int(i*fps)+1 for i in range(10)]  # indices of frames to keep in each chunk
    return chunks, frame_indices

# function to create dataframe with video data
def create_dataframe(video_urls):
    data = {'video_id': [], 'video_url': [], 'video_parts': [], 'frame_rate': []}
    for video_url in video_urls:
        video_id = video_url.split('=')[-1]
        video_path = f'{video_id}.mp4'
        #os.system(f'youtube-dl {video_url} -o "{video_path}"')  # download video
        chunks, frame_indices = split_video(video_path)
        for i, chunk in enumerate(chunks):
            start, end = chunk
            part_path = f'{video_id}-p{i+1}.mp4'
            cap = cv2.VideoCapture(video_path)
            out = cv2.VideoWriter(part_path, cv2.VideoWriter_fourcc(*'mp4v'), 2, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
            
            for j in range(start, end):
                ret, frame = cap.read()
                if not ret:
                    break
                if j in frame_indices:
                    out.write(frame)
            out.release()
            data['video_id'].append(video_id)
            data['video_url'].append(video_url)
            data['video_parts'].append(part_path)
            data['frame_rate'].append(len(frame_indices))  # 2 frames per second
    df = pd.DataFrame(data)
    return df

# create dataframe
video_urls = ["https://www.youtube.com/watch?v=JDwduqfuHjc"]
df = create_dataframe(video_urls)

# save video chunks in current directory
for part_path in df['video_parts']:
    os.system(f'mv {part_path} {os.getcwd()}')

# print dataframe
print(df)

      video_id                                    video_url  \
0  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
1  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
2  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
3  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
4  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
5  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
6  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   

          video_parts  frame_rate  
0  JDwduqfuHjc-p1.mp4          10  
1  JDwduqfuHjc-p2.mp4          10  
2  JDwduqfuHjc-p3.mp4          10  
3  JDwduqfuHjc-p4.mp4          10  
4  JDwduqfuHjc-p5.mp4          10  
5  JDwduqfuHjc-p6.mp4          10  
6  JDwduqfuHjc-p7.mp4          10  


In [32]:
import os
import pandas as pd
import cv2

# Define function to split video into 10-second chunks and convert to 2fps
def split_video(video_url, output_folder):
    # Load video
    cap = cv2.VideoCapture("JDwduqfuHjc.mp4")
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    # Define parameters for splitting
    chunk_size = 10 * fps  # split video into 10-second chunks
    frame_rate = 2  # keep only first and middle frame of each chunk

    # Define output file format
    output_format = f"{os.path.splitext(os.path.basename(video_url))[0]}-p{{}}.mp4"

    # Split video and convert to desired frame rate
    chunk_num = 1
    while True:
        # Read chunk of video
        ret, frame = cap.read(chunk_size)
        if not ret:
            break  # end of video reached

        # Convert chunk to 2fps and save to file
        chunk_duration = min(chunk_size / fps, 10)  # duration of chunk in seconds
        output_file = os.path.join(output_folder, output_format.format(chunk_num))
        writer = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*"mp4v"), frame_rate, (frame_width, frame_height))
        for i in range(int(chunk_duration * frame_rate)):
            if i == 0 or i == int(chunk_duration * frame_rate) // 2:
                writer.write(frame[int(i * fps / frame_rate)])  # keep only first and middle frame of each chunk
        writer.release()

        # Move to next chunk
        chunk_num += 1

    # Release resources
    cap.release()

    return chunk_num - 1


# Define input data
video_links = [
    "https://www.youtube.com/watch?v=JDwduqfuHjc"
]

# Define output folder
output_folder = os.getcwd()

# Initialize dataframe
data = {
    "video_id": [],
    "video_url": [],
    "video_parts": [],
    "frame_rate": [],
}
df = pd.DataFrame(data)

# Process each video
for video_url in video_links:
    # Get video ID
    video_id = os.path.splitext(os.path.basename(video_url))[0]

    # Split video into chunks and save data to dataframe
    num_chunks = split_video(video_url, output_folder)
    video_parts = [f"{video_id}-p{i}.mp4" for i in range(1, num_chunks + 1)]
    frame_rate = [2] * num_chunks
    data = {
        "video_id": [video_id] * num_chunks,
        "video_url": [video_url] * num_chunks,
        "video_parts": video_parts,
        "frame_rate": frame_rate,
    }
    df = df.append(pd.DataFrame(data), ignore_index=True)

# Print dataframe
print(df)


error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'VideoWriter'
> Overload resolution failed:
>  - Can't parse 'frameSize'. Sequence item with index 0 has a wrong type
>  - VideoWriter() missing required argument 'frameSize' (pos 5)
>  - VideoWriter() missing required argument 'params' (pos 5)
>  - VideoWriter() missing required argument 'frameSize' (pos 5)


In [26]:
df

,video_id,video_url,video_part,frame_rate


In [34]:
import pandas as pd
import cv2
import os

# List of YouTube video URLs
video_urls = [
    "https://www.youtube.com/watch?v=JDwduqfuHjc"
]

# Create an empty list to store the data for each video
video_data = []

# Loop through each video URL
for url in video_urls:
    # Get the video ID from the URL
    video_id = url.split("=")[-1]
    # Download the video using youtube-dl
    os.system(f"youtube-dl -f 'bestvideo[ext=mp4]' -o 'video.mp4' {url}")
    # Open the video using OpenCV2
    cap = cv2.VideoCapture("video.mp4")
    # Get the total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Get the frame rate
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    # Split the video into 10-second chunks with two frames per second
    chunk_size = 10 * frame_rate
    for i in range(0, total_frames, chunk_size):
        # Set the start and end frames for the chunk
        start_frame = i
        end_frame = min(i + chunk_size, total_frames)
        # Get the chunk of frames
        frames = []
        for j in range(start_frame, end_frame):
            # Read the frame
            ret, frame = cap.read()
            # Keep only the first and middle frame
            if j == start_frame or j == (start_frame + end_frame) // 2:
                frames.append(frame)
        # Save the chunk as an mp4 file
        chunk_filename = f"video{video_id}-p{int(i/chunk_size)+1}.mp4"
        cv2.VideoWriter(f"c:/HYANA/{chunk_filename}", cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (frames[0].shape[1], frames[0].shape[0])).write(frames)
        # Add the video data to the list
        video_data.append({
            "video_id": video_id,
            "video_url": url,
            "video_parts": chunk_filename,
            "frame_rate": frame_rate,
        })
    # Release the OpenCV2 capture object
    cap.release()

# Create the final data frame
df = pd.DataFrame(video_data)
print(df)


error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'write'
> Overload resolution failed:
>  - image is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'image'


In [65]:
import pandas as pd
import cv2
import os

# List of YouTube video URLs
video_urls = [
    "https://www.youtube.com/watch?v=JDwduqfuHjc"
]

# Create an empty list to store the data for each video
video_data = []

def chunker(video_urls_list):
    # Loop through each video URL
    for url in video_urls_list:
        # Get the video ID from the URL
        video_id = url.split("=")[-1]
        # Download the video using youtube-dl
        os.system(f"youtube-dl -f 'bestvideo[ext=mp4]' -o 'video.mp4' {url}")
        # Open the video using OpenCV2
        cap = cv2.VideoCapture("JDwduqfuHjc.mp4")
        # Get the total number of frames
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # Get the frame rate
        frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
        # Split the video into 10-second chunks with two frames per second
        chunk_size = 10 * frame_rate
        for i in range(0, total_frames, chunk_size):
            # Set the start and end frames for the chunk
            start_frame = i
            end_frame = min(i + chunk_size, total_frames)
            # Get the chunk of frames
            frames = []
            for j in range(start_frame, end_frame):
                # Read the frame
                ret, frame = cap.read()
                # Keep only the first and middle frame
                if j == start_frame or j == (start_frame + end_frame) // 2:
                    frames.append(frame)
            # Save the chunk as an mp4 file
            chunk_filename = f"{video_id}-p{int(i/chunk_size)+1}.mp4"

                # path to input video
            part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
            os.makedirs(part_fpath, exist_ok = True)
            main_video_path =f'{os.path.join(part_fpath, chunk_filename)}'


            cf = r"C:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts"
            frame_rate_chunk = 2
            out = cv2.VideoWriter(f"{main_video_path}", cv2.VideoWriter_fourcc(*'mp4v'), frame_rate_chunk, (frames[0].shape[1], frames[0].shape[0]))
            for frame in frames:
                out.write(frame)
            out.release()
            # Add the video data to the list
            video_data.append({
                "video_id": video_id,
                "video_url": url,
                "video_parts": chunk_filename,
                "frame_rate": frame_rate_chunk,
            })
        # Release the OpenCV2 capture object
        cap.release()


chunker(video_urls)
# Create the final data frame
df = pd.DataFrame(video_data)
print(df)


      video_id                                    video_url  \
0  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
1  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
2  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
3  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
4  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
5  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
6  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   

          video_parts  frame_rate  
0  JDwduqfuHjc-p1.mp4           2  
1  JDwduqfuHjc-p2.mp4           2  
2  JDwduqfuHjc-p3.mp4           2  
3  JDwduqfuHjc-p4.mp4           2  
4  JDwduqfuHjc-p5.mp4           2  
5  JDwduqfuHjc-p6.mp4           2  
6  JDwduqfuHjc-p7.mp4           2  


In [56]:
df

,video_id,video_url,video_parts,frame_rate
0,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p1.mp4,2
1,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p2.mp4,2
2,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p3.mp4,2
3,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p4.mp4,2
4,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p5.mp4,2
5,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p6.mp4,2
6,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p7.mp4,2


In [ ]:


import base64

def 
# add columns for each frame
for i in range(10):
    df[f'frame_{i+1}'] = ''

# iterate over each row and extract frames
for index, row in df.iterrows():
    video_parts = row['video_parts']
    frames = []
    for i in range(10):
        frame_path = f'{video_parts[:-4]}-frame{i+1}.jpg'
        #frame_path = f'/home/siddharthsk7t/DL/DL-temp/frames/frame{i+1}.jpg'

        frame = cv2.imread(frame_path)
        byte_string = image_to_byte_string(frame)
        frames.append(byte_string)
        os.remove(frame_path)
    df.loc[index, df.columns[4:]] = frames

